In [2]:
import requests
import json
import pandas as pd
from datetime import datetime
import mysql.connector


In [3]:
mydb = mysql.connector.connect(
    host="localhost",
    user="root",
    password="admin",
    database="sapo_data",
    charset="utf8mb4"
)

In [4]:
url = "https://yoururl/admin/orders.json?page=1&limit=100"
cookie = "report_ver=3; _fw_crm_v=b03a9b5b-9cdb-4d81-ab85-c51c8076a2ee; _fbp=fb.1.1722010576000.953081039792325240; _gid=GA1.2.421020518.1727614808; _gat_gtag_UA_66880228_3=1; sapo-admin=CfDJ8DA1016STPdNrAaTdK2HgWfxhsoRYw9VbP9VUnDSNE2WlEuyslidCqjA_1_a1xy_5a4YQKJLlxeoGjapLLCfdUKaFJ46-X2GTWpIP7rVxkNBXC6J7aQO4ca86AWBoC2d0bHw7f2_eyt4UbnNR3mmOfPN3J_6LG8p1g_fVgfezt4I7vXr_LLVGHlzeaEBh5JKBf3-fkJHdb7tJiPpM-t4VR5DQHyakY3x1Xg_I9fa_bx1uOWFe2bBeMfkYNms2Dg5G3oBECIP-O9dAmq4tEnbSk8xngForMznApmVKJ2uDRIWotdIkPF5wRTVNnaplltcemGIvKaE7dyCPGSp1WSWn2A0Bjv0U3_sRyJha3xDOXOwdhJNzNYW8F5gCvXMX3noYyGvxfE-NaJZbG9MbNZCMQRFafmNwz-A093VKeJMl0z5VmgEbLSNgPyqut05dK09GVkTnbmbVdpQBiCm-YYnVUHa_4hTIlu3_n_iT3iwmpT98QkXUMkTpoF4954-vb8loakjCnvnnW0d2PTp8LYh-ePYKy6ojtACwDfeRjCEHITEN5dWclgg1TVuJ10lqlQO_971xkxKJGJs3SnvuJEM7q0NWnTrnbKfsRdlgXU5ZuMiHXmG2GxyGFPSafpOdu88-w; lang=vi; _admin_session_id=c38ca72b-1a3d-e873-1c3c-4c8ea96935ab; .AspNetCore.Session=CfDJ8DA1016STPdNrAaTdK2HgWd9y3RxPdMw8o54%2FWOyVzz81BddsGzevkYFPtRiLQGSiEzRRMoCGQ6RBMq3eLB78Ri3KEcaXUADmkxXfNQmCOXp2pbddnJIcWJjWUKh9t31QBOtgR8nZQqTBrU9AL8nblfEFdFf7Jwlfmk%2F2Rr3YeMF; _ga_4NX0F91DEX=GS1.1.1727700346.57.1.1727700348.0.0.0; _ga=GA1.1.1357561029.1697356741; _ga_Z1V9QDBF2Q=GS1.1.1727700346.57.1.1727700349.0.0.0"
payload = {}
headers = {
  'Cookie': cookie,
}

response = requests.request("GET", url, headers=headers, data=payload)


In [227]:
# load data
data: dict = json.loads(response.text)

In [5]:
created_ons = []
codes = []
customer_codes = []
customer_names = []
skus = []
product_names = []
lots_numbers = []
dates = []
units = []
quantities = []
gifts = []
prices_sapo = []
prices = []
prices_before_tax = []
stocks = []
discount_sales = []
discount_gifts = []
totals = []
unit_prices = []
paids = []
notes = []

In [6]:
def insert_order_database(
    created_on      ,
    code            ,
    customer_code   ,
    customer_name   ,
    sku             ,
    product_name    ,
    lot_number      ,
    date            ,
    unit            ,
    quantity        ,
    gift            ,
    price_sapo      ,
    price           ,
    price_before_tax,
    stock           ,
    discount_sale   ,
    discount_gift   ,
    total           ,
    unit_price      ,
    paid            ,
    note            ,
):
    mycursor = mydb.cursor()
    sql = "INSERT IGNORE  INTO `order` (created_on, code, customer_code, customer_name, sku, product_name, lot_number, date, unit, quantity, gift, price_sapo, price, price_before_tax, stock, discount_sale, discount_gift, total, unit_price, paid, note) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
    val = (created_on, code, customer_code, customer_name, sku, product_name, lot_number, date, unit, quantity, gift, price_sapo, price, price_before_tax, stock, discount_sale, discount_gift, total, unit_price, paid, note)
    try:
        mycursor.execute(sql, val)
        
        mydb.commit()
    except:
        raise    

    
    

In [7]:
for order in data["orders"]:
    if order["location_id"] != 598184:
        continue
    created_on = order["created_on"]
    created_date = datetime.strptime(created_on,"%Y-%m-%dT%H:%M:%SZ")
    created_on = datetime.strptime(created_on,"%Y-%m-%dT%H:%M:%SZ").strftime("%d/%m/%Y")
    code = order["code"]
    customer_code = order["customer_data"]["code"]
    customer_name = order["customer_data"]["name"]
    if "Finelife" in customer_name: continue
    paid = True
    if order["payment_status"] == "unpaid":
        paid = False
    stock = "CN HCM"
    for order_item in order["order_line_items"]:
        sku =  order_item["sku"]
        product_name = order_item["product_name"]
        unit = order_item["unit"]
        gift = 0
        price_sapo = order_item["price"]
        price_before_tax = price_sapo / 1.08
        price_before_tax = round(price_before_tax, 2)
        price = 0
        discount_sale = order_item["discount_rate"]    
        for lot in order_item["lots_dates"]:
            lots_number = lot["lots_number"]
            manufacture_date = datetime.strptime(lot["manufacture_date"],"%Y-%m-%dT%H:%M:%SZ").strftime("%d/%m")
            expiration_date = datetime.strptime(lot["expiration_date"],"%Y-%m-%dT%H:%M:%SZ").strftime("%d/%m")
            lots_date = manufacture_date + "-" + expiration_date
            quantity = int(lot["quantity"])
            total = quantity * price_sapo * (1-discount_sale/100)
            discount_gift = 0
            if total == 0:
                discount_gift = 100
                discount_sale = 0
            unit_price = total/quantity
            # save data 
            created_ons.append(created_on)
            codes.append(code)
            customer_codes.append(customer_code)
            customer_names.append(customer_name)
            skus.append(sku)
            product_names.append(product_name)
            lots_numbers.append(lots_number)
            dates.append(lots_date)
            units.append(unit)
            quantities.append(quantity)
            gifts.append(gift)
            prices_sapo.append(price_sapo)
            prices.append(price)
            prices_before_tax.append(price_before_tax)
            stocks.append(stock)
            discount_sales.append(discount_sale)
            discount_gifts.append(discount_gift)
            totals.append(total)
            unit_prices.append(unit_price)
            paids.append(paid)
            notes.append("")
            insert_order_database(created_date, code, customer_code, customer_name, sku, product_name, lots_number, lots_date, unit, quantity, gift, price_sapo, price, price_before_tax, stock, discount_sale, discount_gift, total, unit_price, paid, "")

NameError: name 'data' is not defined

In [231]:

data = {
    "created_on": created_ons,
    "code": codes,
    "customer_code": customer_codes,
    "customer_name": customer_names,
    "sku": skus,
    "product_name": product_names,
    "lots_number": lots_numbers,
    "date": dates,
    "unit": units,
    "quantity": quantities,
    "gift": gifts,
    "price_sapo": prices_sapo,
    "price": prices,
    "price_before_tax": prices_before_tax,
    "stock": stocks,
    "discount_sale": discount_sales,
    "discount_gift": discount_gifts,
    "total": totals,
    "unit_price": unit_prices,
    "paid": paids,
    "note": notes
}

# Create a DataFrame
df = pd.DataFrame(data)

In [232]:
df.to_csv('output.csv', index=False)

In [8]:
mydb.close()